In [5]:
import pandas
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from scipy.optimize import minimize

In [ ]:
def mixer(preds, weights=None, geo=False):
    
    if weights is None:
        weights = [1.0/len(preds) for i in range(len(preds))]
        
    if type(weights) is float:
        weights = [weights for i in range(len(preds))]
        
    assert len(weights) == len(preds)
        
    if geo:
        prod = 1.0
        
        for pred, weight in zip(preds, weights):
            prod *= pred**weight
            
        return prod ** (1/reduce(lambda x, y: x + y, weights))
    
    else:
        suma = 0.0
        
        for pred, weight in zip(preds, weights):
            suma += pred*weight
            
        return suma/reduce(lambda x, y: x + y, weights)

In [2]:
d = pandas.read_csv('../Data/train_folds.csv')

In [7]:
y = LabelEncoder().fit_transform(d['target'].iloc[d['test_fold'].values == 0])

In [8]:
d1 = pandas.read_csv('valPred_Fernando_Lasagne1_CV046100.csv')
d2 = pandas.read_csv('valPred_Fernando_Lasagne2_CV04611.csv')
d3 = pandas.read_csv('valPred_Fernando_Lasagne3_CV0467.csv')
d4 = pandas.read_csv('valPred_Fernando_Lasagne4_CV04684.csv')

In [18]:
pred_list = [d1.values[:, 1:], d2.values[:, 1:], d3.values[:, 1:], d4.values[:, 1:]]

In [20]:
opt = minimize(lambda x: log_loss(y, mixer(pred_list, weights=x, geo=False)),
               x0=[1] * 4, bounds=[(0, None), (0, None), (0, None), (0, None)])

opt

  status: 0
 success: True
    nfev: 30
     fun: 0.44180415992337757
       x: array([ 0.66080847,  1.19588716,  1.3012324 ,  0.64625372])
 message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     jac: array([  9.99200722e-08,  -5.55111512e-08,   2.33146835e-07,
        -4.77395901e-07])
     nit: 5

In [21]:
preds = mixer(pred_list, weights=opt.x, geo=False)

print('run1: ', log_loss(y, d1.values[:, 1:]))
print('run2: ', log_loss(y, d2.values[:, 1:]))
print('run3: ', log_loss(y, d3.values[:, 1:]))
print('run4: ', log_loss(y, d4.values[:, 1:]))
print('mix:  ', log_loss(y, preds))

('run1: ', 0.46101432250651986)
('run2: ', 0.46111063173373545)
('run3: ', 0.46988520315683835)
('run4: ', 0.46841301720980577)
('mix:  ', 0.44180415992337757)
